In [5]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
import pyiqa
from sklearn.model_selection import train_test_split

In [6]:

# ── USER CONFIG: ────────────────────────────────────
dataset_dir= "Datasets/"

output_dir= "G:/MaestriaInformatica/Tesis/Outputs/Bicubic/"

LR_DIR      = dataset_dir +"SatImages/perusat_v5_rgb/LR_BICUBIC/X2/"      # carpeta con imágenes LR (256×256)
HR_DIR      = dataset_dir +"SatImages/perusat_v5_rgb/HR/"     # carpeta con imágenes HR (p.ej. 512×512)
SUFFIX      = "_x2"                 # sufijo de los nombres LR
HR_HEIGHT   = 512                   # alto de las imágenes HR
HR_WIDTH    = 512                   # ancho de las imágenes HR
BATCH_SIZE  = 8
MAX_IMAGES  = 0     # 0 = usar todas
VAL_FRAC    = 0.2   # fracción para validación
DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ─────────────────────────────────────────────────────────────────────────────

# ── examples config ─────────────────────────────────────────────────────
EXAMPLE_NAMES = [
    "IMG_PER1_20161203152919_ORT_MS_000670_1024-3072.png", "IMG_PER1_20161203152919_ORT_MS_000670_2048-512.png", "IMG_PER1_20170422154946_ORT_MS_000041_3072-4608.png", "IMG_PER1_20170422154946_ORT_MS_000041_1536-2560.png", "IMG_PER1_20170422154946_ORT_MS_000041_512-3584.png",
    "IMG_PER1_20161203152919_ORT_MS_000670_4608-4608.png", "IMG_PER1_20170422154946_ORT_MS_000659_1536-4608.png", "IMG_PER1_20170422154946_ORT_MS_000659_5120-3584.png", "IMG_PER1_20170422154946_ORT_MS_001277_3584-4608.png", "IMG_PER1_20190320154045_ORT_MS_000041_4608-3072.png",
]
EXAMPLES_DIR  = output_dir + "samples/"  # donde se guardan los 10 SR ejemplos


In [7]:
def load_dataset(hr_dir, lr_dir, suffix, hr_size, max_images=100, val_frac=0.1):
    all_hr = [f for f in sorted(os.listdir(hr_dir))
              if f.lower().endswith(('.png','.jpg','.jpeg'))]
    valid_hr = []
    for fn in all_hr:
        with Image.open(os.path.join(hr_dir, fn)) as im:
            if im.size == (hr_size[1], hr_size[0]):
                valid_hr.append(fn)

    total_hr = len(valid_hr)
    if max_images > 0 and total_hr < max_images:
        raise RuntimeError(f"Sólo hay {total_hr} imágenes HR de tamaño {hr_size}, "
                           f"pero pediste max_images={max_images}.")
    if max_images == 0:
        max_images = total_hr

    selected_hr = valid_hr[:max_images]
    lr_names = []
    for hr_fn in selected_hr:
        base, ext = os.path.splitext(hr_fn)
        lr_fn = base + suffix + ext
        lr_path = os.path.join(lr_dir, lr_fn)
        if not os.path.exists(lr_path):
            raise RuntimeError(f"No existe LR para '{hr_fn}' → buscado '{lr_fn}'.")
        with Image.open(lr_path) as im:
            expected = (hr_size[1]//2, hr_size[0]//2)
            if im.size != expected:
                raise RuntimeError(
                    f"LR '{lr_fn}' tiene tamaño {im.size}, pero esperaba {expected}."
                )
        lr_names.append(lr_fn)

    train_hr, val_hr, train_lr, val_lr = train_test_split(
        selected_hr, lr_names, test_size=val_frac, random_state=42
    )

    to_tensor = transforms.ToTensor()
    def make_dataset(hr_list, lr_list):
        hr_tensors, lr_tensors = [], []
        for hr_fn, lr_fn in zip(hr_list, lr_list):
            hr = Image.open(os.path.join(hr_dir, hr_fn)).convert('RGB')
            lr = Image.open(os.path.join(lr_dir, lr_fn)).convert('RGB')
            hr_tensors.append(to_tensor(hr))
            lr_tensors.append(to_tensor(lr))
        return torch.stack(lr_tensors), torch.stack(hr_tensors)

    lr_train, hr_train = make_dataset(train_hr, train_lr)
    lr_val,   hr_val   = make_dataset(val_hr,   val_lr)

    return TensorDataset(lr_train, hr_train), TensorDataset(lr_val, hr_val)

def evaluate_dataset(generator, data_loader, device):
    generator.eval()
    total_psnr = total_ssim = 0.0
    count = 0

    ssimc_metric = pyiqa.create_metric('ssimc', device=device)
    psnr_metric  = pyiqa.create_metric('psnr',  device=device)

    with torch.no_grad():
        for lr_imgs, hr_imgs in data_loader:
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            sr_imgs = generator(lr_imgs)
            ssim_val = ssimc_metric(hr_imgs, sr_imgs)[0].item()
            psnr_val = psnr_metric(hr_imgs, sr_imgs)[0].item()

            total_psnr += psnr_val * hr_imgs.size(0)
            total_ssim += ssim_val * hr_imgs.size(0)
            count += hr_imgs.size(0)

    return (total_psnr / count) if count else 0, (total_ssim / count) if count else 0

In [8]:

# ── Bicubic “Generator” ──────────────────────────────────────────────────────

class BicubicGenerator(nn.Module):
    def __init__(self, scale_factor=2):
        super().__init__()
        self.scale = scale_factor
    def forward(self, x):
        return F.interpolate(x, scale_factor=self.scale,
                             mode="bicubic", align_corners=False)

In [9]:
# ── Main: carga datos, evalúa y muestra resultados ────────────────────────────

if __name__ == "__main__":
    # 1) Prepara validación
    _, val_dataset = load_dataset(
        hr_dir     = HR_DIR,
        lr_dir     = LR_DIR,
        suffix     = SUFFIX,
        hr_size    = (HR_HEIGHT, HR_WIDTH),
        max_images = MAX_IMAGES,
        val_frac   = VAL_FRAC
    )
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # 2) Instancia el generador bicúbico
    generator = BicubicGenerator(scale_factor=2).to(DEVICE)

    # 3) Evalúa PSNR/SSIM
    avg_psnr, avg_ssim = evaluate_dataset(generator, val_loader, DEVICE)
    print(f"→ PSNR promedio: {avg_psnr:.2f} dB")
    print(f"→ SSIM promedio: {avg_ssim:.4f}")

    # 4) Génère 10 ejemplos y guárdalos
    os.makedirs(EXAMPLES_DIR, exist_ok=True)
    for hr_fn in EXAMPLE_NAMES:
        base, ext = os.path.splitext(hr_fn)
        lr_fn = base + SUFFIX + ext
        lr_path = os.path.join(LR_DIR, lr_fn)

        # Carga y convierte
        lr_img = Image.open(lr_path).convert("RGB")
        lr_tensor = transforms.ToTensor()(lr_img).unsqueeze(0).to(DEVICE)

        # Upsample
        with torch.no_grad():
            sr_tensor = generator(lr_tensor)

        # A PIL y guardar
        sr_img = TF.to_pil_image(sr_tensor.squeeze(0).cpu().clamp(0,1))
        save_name = f"{base}_bicubic{ext}"
        sr_img.save(os.path.join(EXAMPLES_DIR, save_name))
        print(f"Guardado ejemplo: {save_name} → {EXAMPLES_DIR}")

→ PSNR promedio: 40.51 dB
→ SSIM promedio: 0.9622
Guardado ejemplo: IMG_PER1_20161203152919_ORT_MS_000670_1024-3072_bicubic.png → G:/MaestriaInformatica/Tesis/Outputs/Bicubic/samples/
Guardado ejemplo: IMG_PER1_20161203152919_ORT_MS_000670_2048-512_bicubic.png → G:/MaestriaInformatica/Tesis/Outputs/Bicubic/samples/
Guardado ejemplo: IMG_PER1_20170422154946_ORT_MS_000041_3072-4608_bicubic.png → G:/MaestriaInformatica/Tesis/Outputs/Bicubic/samples/
Guardado ejemplo: IMG_PER1_20170422154946_ORT_MS_000041_1536-2560_bicubic.png → G:/MaestriaInformatica/Tesis/Outputs/Bicubic/samples/
Guardado ejemplo: IMG_PER1_20170422154946_ORT_MS_000041_512-3584_bicubic.png → G:/MaestriaInformatica/Tesis/Outputs/Bicubic/samples/
Guardado ejemplo: IMG_PER1_20161203152919_ORT_MS_000670_4608-4608_bicubic.png → G:/MaestriaInformatica/Tesis/Outputs/Bicubic/samples/
Guardado ejemplo: IMG_PER1_20170422154946_ORT_MS_000659_1536-4608_bicubic.png → G:/MaestriaInformatica/Tesis/Outputs/Bicubic/samples/
Guardado ejemp